In [ ]:
import sys
sys.path.append("/home/idumeur/code") #To modify
sys.path.append("/home/idumeur/code/cGAN_sent2_sim/") #to Modify

In [ ]:
import utils.display_image as d
import pandas as pd
from utils.load_dataset import load_data,load_from_dir
from utils.display_image import one_band_hist,plot_one_band,display_final_tile
import matplotlib.pyplot as plt
from utils.image_find_tbx import find_image_indir
import numpy as np
from change_detection.detection import ACP_on_batch
from change_detection.display_detection import display_detection
from utils.normalize import load_dict_scaler,rescale_array
from utils.metrics import compute_metric
import os

In [ ]:
# ADAPT GIVEN YOUR OWN FILE HIERARCHY
# To run this script you have run a prediction for a trained model beforehand.
GLOBAL_PATH="/srv/osirim/idumeur/data/dataset6/prepro1/input_large_dataset/"
PATH=os.path.join(GLOBAL_PATH,"train/")
PATH_TEST=os.path.join(GLOBAL_PATH,"test/")
PATH_SAVE="/srv/osirim/idumeur/data/dataset6/prepro1/input_large_dataset/train_change_detection3/" #path where information for change detection model : EVS + kmeans model
NB_BAND_X=8
NB_BAND_LABEL=4
PATH_DIR=PATH+"dataX/"
PATH_TRAININGS="/srv/osirim/idumeur/trainings/" 
NAME_MODEL="new_model_corr" #name of the model
TRAINING_NBER="42" #index of the training from which you want to load the prediction

PATH_THIS_MODEL=PATH_TRAININGS+NAME_MODEL+"/" 
PATH_THIS_TRAINING="{}{}/training_{}/".format(PATH_TRAININGS,NAME_MODEL,TRAINING_NBER) #path to the directory which contains checkpoints directory
PATH_SCALER=PATH_THIS_TRAINING+"scaler/"
PATH_PRED="{}image_test_iter_690/".format(PATH_THIS_TRAINING) #Path where predicted image are saved. (Check predict methods)
PATH_CHECKPOINT="{}checkpoints/".format(PATH_THIS_TRAINING)
PATH_SAVED_IM="{}saved_training_images/".format(PATH_THIS_TRAINING)

## PARAMETERS FOR DATA (not recommended to change)
DICT_GROUP_BAND_LABEL={"R,G,B":[0,1,2],"NIR":[3]}
DICT_GROUP_BAND_X={"VV":[0,2],"VH":[1,3],"R,G,B":[4,5,6],"NIR":[7]}
S2_BANDS=['R,G,B', 'NIR']
S1_BANDS=['VV', 'VH']
FACT_STD_S2=1/7 
FACT_STD_S1=1/5
MAX_DISPLAY=1
DICT_RESCALE_TYPE={"VV": "StandardScaler", "VH":"StandardScaler","R,G,B":"StandardScaler","NIR":"StandardScaler"}

In [ ]:
# If not done before : run the change detection method on the train dataset
data_X_train,data_y_train,l_dict_train=load_data(PATH, x_shape=(256,256,NB_BAND_X), label_shape=(256,256,NB_BAND_LABEL), normalization=False,dict_band_X=None,dict_band_label=None,dict_rescale_type=None,lim=None)
r_d0_val=data_X_train[:,:,:,[4]]
r_d1_val=data_y_train[:,:,:,[0]]
dic_band_name={"red":[[4],[0]],"green":[[5],[1]],"blue":[[6],[2]],"nir":[[7],[3]]}
for band_name in dic_band_name:
    print("ACP on {}".format(band_name))
    _,_, _=ACP_on_batch(data_X_train[:,:,:,dic_band_name[band_name][0]],data_y_train[:,:,:,dic_band_name[band_name][1]], kernel_dim=4, n_components=3, k=2, padding="symmetric",band=None,path_save=PATH_SAVE,load_dir=None,band_name=band_name)
    

# GROUND TRUTH DATA
The EVS and k_means algorithm for each band were saved in PATH_SAVE.Thus we load the change map on the validation batch for each bands.
If you haven't run the change detection on the training dataset please do it first (cell 4). Then, run the change detection method loading the change detection algorithm using load_dir. 

In [ ]:
#load the raw data
data_X_val,data_y_val,l_dict_val=load_data(PATH_TEST, x_shape=(256,256,NB_BAND_X), label_shape=(256,256,NB_BAND_LABEL), normalization=False,dict_band_X=None,dict_band_label=None,dict_rescale_type=None,lim=None)

In [ ]:
r_d0_val=data_X_val[:,:,:,[4]]
r_d1_val=data_y_val[:,:,:,[0]]
_,change_map_val_r, batch_clean_change_map_val_r=ACP_on_batch(r_d0_val,r_d1_val, kernel_dim=4, n_components=3, k=2, padding="symmetric",band=None,path_save=None,load_dir=PATH_SAVE,band_name="red")

In [ ]:
r_d0=data_X_val[:,:,:,[5]]
r_d1=data_y_val[:,:,:,[1]]
_,change_map_val_g, batch_clean_change_map_val_g=ACP_on_batch(r_d0,r_d1, kernel_dim=4, n_components=3, k=2, padding="symmetric",band=None,path_save=None,load_dir=PATH_SAVE,band_name="green")

In [ ]:
r_d0=data_X_val[:,:,:,[6]]
r_d1=data_y_val[:,:,:,[2]]
_,change_map_val_b, batch_clean_change_map_val_b=ACP_on_batch(r_d0,r_d1, kernel_dim=4, n_components=3, k=2, padding="symmetric",band=None,path_save=None,load_dir=PATH_SAVE,band_name="blue")

In [ ]:
r_d0=data_X_val[:,:,:,[7]]
r_d1=data_y_val[:,:,:,[3]]
_,change_map_val_nir, batch_clean_change_map_val_nir=ACP_on_batch(r_d0,r_d1, kernel_dim=4, n_components=3, k=2, padding="symmetric",band=None,path_save=None,load_dir=PATH_SAVE,band_name="nir")

# Simulated image

In [ ]:
dict_scaler=load_dict_scaler(input_dir=PATH_SCALER,l_band_group=S2_BANDS+S1_BANDS)

dict_scaler contains the scaler method used on the train data. It has been saved during the training. It will be used to unscaled the ouptut tiles predicted by the generator

In [ ]:
#load the data
pred_batch=np.zeros(data_y_val.shape)
pred_batch.shape
batch_pred,l_final_tiles,_=load_from_dir(PATH_PRED,(256,256,4),lim=496)# a list of path of the images
max_im=len(l_final_tiles)
print(batch_pred.shape)

In [ ]:
#We unscale the predicted tiles
_, pred_batch_unscale, _=rescale_array(data_X_val, batch_pred, dict_group_band_X=DICT_GROUP_BAND_X, dict_group_band_label=DICT_GROUP_BAND_LABEL,
                  dict_rescale_type=DICT_RESCALE_TYPE, s1_log=True, dict_scale=dict_scaler, invert=True, s2_bands=S2_BANDS,
                  s1_bands=S1_BANDS, fact_scale2=FACT_STD_S2, fact_scale1=FACT_STD_S1, clip_s2=True)

# Global metrics
We compute PSNR, SSIM and SAM bewteen the predicted batch and the ground truth batch.

In [ ]:
from skimage.measure import compare_ssim as ssim
from utils.metrics import calculate_psnr
metric_name=["psnr","nchangedpix","mse"]

def compute_metrics(vector1,vector2,metrics=None):
    if metrics is None:
        metrics=metric_name
    #print(vector1.shape)
    #print(vector2.shape)
    psnr=calculate_psnr(vector1,vector2, max_value=1.27) #we divide its value by the nber of pixels
    mse=np.mean((np.array(vector1, dtype=np.float32) - np.array(vector2, dtype=np.float32)) ** 2)
    #sim_val=ssim(vector1.ravel(),vector2.ravel())
    dict_metric=dict(zip(metrics,[psnr,vector1.shape[0],mse]))
    print(dict_metric)
    return dict_metric

In [ ]:
_, metric_val=compute_metric(pred_batch_unscale,data_y_val,compute_sam=True,max_value=1.27)

In [ ]:
print(dict(zip(["psnr","sim","sam"],metric_val)))

In [ ]:
df=pd.DataFrame(columns=metric_name)
bands_name=["Red","Green","Blue","NIR"]
for i in range(4):
        cp_batch_gt=data_y_val[:,:,:,[i]][l_clean_cm[i]>=0]
        cp_batch_gt_d0=data_X_val[:,:,:,[i+3]][l_clean_cm[i]>=0]
        cp_batch_sim=pred_batch_unscale[:,:,:,[i]][l_clean_cm[i]>=0]
        dict_metric=compute_metrics(cp_batch_sim,cp_batch_gt)
        dict_metric2=compute_metrics(cp_batch_gt_d0,cp_batch_gt)
        #dict_metric=dict(zip(metric_name,metric_val))
        dict_metric.update({"band":format(bands_name[i]),"type":"gt vs simulated"})
        dict_metric2.update({"band":format(bands_name[i]),"type":"gt day 0 vs day 1"})
        df=df.append(dict_metric,ignore_index=True)
        df=df.append(dict_metric2,ignore_index=True)


In [ ]:
df.columns #index 0 stands for band red, 1 band green, 2 band blue 3 band nir.
df.set_index(["band","type"])

# Metric on changed pixels

In [ ]:
df=pd.DataFrame(columns=metric_name)
bands_name=["Red","Green","Blue","NIR"]
metric_name=["psnr","nchangedpix","mse"]
for i in range(4):
        cp_batch_gt=data_y_val[:,:,:,[i]][l_clean_cm[i]==255]
        cp_batch_gt_d0=data_X_val[:,:,:,[i+3]][l_clean_cm[i]==255]
        cp_batch_sim=pred_batch_unscale[:,:,:,[i]][l_clean_cm[i]==255]
        dict_metric=compute_metrics(cp_batch_sim,cp_batch_gt)
        dict_metric2=compute_metrics(cp_batch_gt_d0,cp_batch_gt)
        #dict_metric=dict(zip(metric_name,metric_val))
        dict_metric.update({"band":format(bands_name[i]),"type":"gt vs simulated"})
        dict_metric2.update({"band":format(bands_name[i]),"type":"gt day 0 vs day 1"})
        df=df.append(dict_metric,ignore_index=True)
        df=df.append(dict_metric2,ignore_index=True)


In [ ]:
df.columns #index 0 stands for band red, 1 band green, 2 band blue 3 band nir.
df.set_index(["band","type"])

# Metrics on unchanged pixels

In [ ]:
# Metrics on unchanged pixels
df_uncp=pd.DataFrame(columns=["psnr","nunchangedpix","mse"])
bands_name=["Red","Green","Blue","NIR"]
for i in range(4):
        cp_batch_gt=data_y_val[:,:,:,[i]][l_clean_cm[i]==0]
        cp_batch_gt_d0=data_X_val[:,:,:,[i+3]][l_clean_cm[i]==0]
        cp_batch_sim=pred_batch_unscale[:,:,:,[i]][l_clean_cm[i]==0]
        dict_metric=compute_metrics(cp_batch_sim,cp_batch_gt,metrics=["psnr","nunchangedpix","mse"])
        dict_metric2=compute_metrics(cp_batch_gt_d0,cp_batch_gt,metrics=["psnr","nunchangedpix","mse"])
        #dict_metric=dict(zip(metric_name,metric_val))
        dict_metric.update({"band":format(bands_name[i]),"type":"gt vs simulated"})
        dict_metric2.update({"band":format(bands_name[i]),"type":"gt day 0 vs day 1"})
        df_uncp=df_uncp.append(dict_metric,ignore_index=True)
        df_uncp=df_uncp.append(dict_metric2,ignore_index=True)


In [ ]:
df_uncp.columns #index 0 stands for band red, 1 band green, 2 band blue 3 band nir.
df_uncp.set_index(["band","type"])

In [ ]:
print(np.max(data_y_val[0,0,0,0]))